# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr').getOrCreate()
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [3]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                            outputCol = 'features')
final_data = assembler.transform(data).select('features', 'Churn')
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [4]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features', labelCol = 'Churn').fit(train_data)
train_summary = model.summary
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                643|                643|
|   mean|0.16018662519440124|0.12286158631415241|
| stddev|0.36706459065798897|  0.328533830368419|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [5]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
preds_and_labels = model.evaluate(test_data)
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'Churn')
AUC = my_eval.evaluate(preds_and_labels.predictions)
AUC

0.7303951367781154

In [13]:
### New data
final_lr_model = LogisticRegression(featuresCol = 'features', labelCol = 'Churn').fit(final_data)
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)
test_new_customers = assembler.transform(new_customers)
final_results = final_lr_model.transform(test_new_customers)
final_results.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       1.0|
+----------+

